In [1]:
import numpy as np
import xarray as xr

In [2]:
# # Experiment to open all datasets and place them immediately into one MEGA dataset.
# # Objective is to write less code :)

# varName = ['PRECT', 'VBOT', 'UBOT']
# hcastDayStr = ['05', '10', '15', '20']
# hcastDay = [5,10,15,20]
# fn = 'tratl_cam5.3_ne30_rneale.globe.fcast.day'

# _dataRaw = [[0,0,0],[0,0,0],[0,0,0],[0,0,0]] # 4 rows (hindcasts) and 3 columns (variables)

# for ii, day in enumerate(hcastDayStr):
#     for jj, _var in enumerate(varName):
# #         print(dri+fn+day+'.'+name+'.nc')
#         _dataRaw[ii][jj] = xr.open_dataset(dri+fn+day+'.'+_var+'.nc', decode_times=False)

# dataRaw = xr.Dataset({'dataRaw':(['hcastDay', 'varName'], _dataRaw)}, 
#                      coords={'hcastDay':hcastDay, 'varName': varName})
# dataRaw.sel(hcastDay=10,varName='VBOT')['dataRaw']

In [53]:
## Import data

## CAM Hindcast data will be imported into a list. Each index represents the number of hindcast day

# Setup strings to be used to create the path to each file
hcastDayStr = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
# hcastDayStr = ['05','10','15','20']
fn = 'tratl_cam5.3_ne30_rneale.globe.fcast.day'
dri = '/home/jlarson1/data/CAM_hindcasts/'

# Initialize empty list that will contain all the specified hindcast days
dataPRECT   = [None] * (len(hcastDayStr)+1)
dataVBOT3hr = [None] * (len(hcastDayStr)+1)
dataUBOT3hr = [None] * (len(hcastDayStr)+1)

# Iterate through the string that dictates which hindcast days will be loaded
for ii,day in enumerate(hcastDayStr):
#     print(dri+fn+day+'.PRECT.nc')
    dataPRECT[  ii+1] = xr.open_dataset(dri+fn+day+'.PRECT.nc', decode_times=False)
    dataVBOT3hr[ii+1] = xr.open_dataset(dri+fn+day+'.VBOT.nc' , decode_times=False)
    dataUBOT3hr[ii+1] = xr.open_dataset(dri+fn+day+'.UBOT.nc' , decode_times=False)
    
## Observation precipitation

# driObs = '/lss/research/agon-lab/TMPA-daily/'
driPRECTObs = '/home/jlarson1/data/'
fnPRECTObs  = 'pr_daily_2009-2010.nc'
dataPRECTObs = xr.open_dataset(driPRECTObs + fnPRECTObs, decode_times=False)

## Observation surface winds

# Note that these datasets were created specifically for this set of hindcasts. 
# For future reference, if a more generic dataset(s) is used, the code in the next few sections might be useful
driWindObs = '/home/jlarson1/data/ERA5/'
fnUBOTObs = 'u10m_2009-2010-partial_daily.nc' ; fnVBOTObs = 'v10m_2009-2010-partial_daily.nc'
dataUBOTObs = xr.open_dataset(driWindObs + fnUBOTObs, decode_times=False)
dataVBOTObs = xr.open_dataset(driWindObs + fnVBOTObs, decode_times=False)

In [4]:
# # Select the obs data from the years of interest (2009 and 2010) that is more or less within the hindcast time bounds
# # Concat together. All this is done to reduce the amount of data and speed the code

# hcastTimeStart = 0   # days since 2009-09-01
# _hcastTimeEnd   = 200 # days since 2009-09-01

# # Adjust end time to fit format of obs data which ends with the decimal .75
# hcastTimeEnd = _hcastTimeEnd - 0.25

# # Select the data that is at the end of 2009 and beginning of 2010 along the time dimension
# dataVBOTObs = xr.concat([dataVBOTObs2009.sel(time=slice((hcastTimeStart+40055)*24,None)),
#                          dataVBOTObs2010.sel(time=slice((hcastTimeEnd+40055)*24))], dim='time')
# dataUBOTObs = xr.concat([dataUBOTObs2009.sel(time=slice((hcastTimeStart+40055)*24,None)),
#                          dataUBOTObs2010.sel(time=slice((hcastTimeEnd+40055)*24))], dim='time')

In [5]:
# # # Time average the 6-hourly wind obs data to get daily wind data

# # Create hindcast day dimension
# hcastDay = [5, 10, 15, 20]

# dataVBOTObsDaily = np.zeros(( len(dataVBOTObs['v10'])//4, len(dataVBOTObs['v10'][0]), len(dataVBOTObs['v10'][0][0]) ))
# dataUBOTObsDaily = np.zeros(( len(dataUBOTObs['u10'])//4, len(dataUBOTObs['u10'][0]), len(dataUBOTObs['u10'][0][0]) ))
# WindObsTime    = np.zeros(( len(dataVBOTObs['v10'])//4 ))

# # Iterate for the number of time stamps in the 6hrly data. Count by 4
# for ii in range (0, len(dataVBOTObs['v10']), 4):
#     dataVBOTObsDaily[ii//4] = dataVBOTObs['v10' ][ii:ii+4].mean(axis=0)
#     dataUBOTObsDaily[ii//4] = dataUBOTObs['u10' ][ii:ii+4].mean(axis=0)
#     # Create new time stamps as "days since 2009-09-01"
#     WindObsTime[ii//4]    = dataVBOTObs['time'][ii:ii+4].mean(axis=0)//24 - 40055

In [6]:
# Time average the 3-hourly wind hindcast data to get daily wind data

# Create an empty array to hold the daily averaged data
vbot = np.zeros([len(hcastDayStr)+1, len(dataVBOT3hr[1]['time'])//8, len(dataVBOT3hr[1]['lat']), len(dataVBOT3hr[1]['lon'])])
ubot = np.zeros([len(hcastDayStr)+1, len(dataUBOT3hr[1]['time'])//8, len(dataUBOT3hr[1]['lat']), len(dataUBOT3hr[1]['lon'])])

# Iterate for the number of hindcast days
for ii in range(1, len(vbot) ):

    # Iterate for the number of time stamps in 3hrly data. Count by 8
    for jj in range(0, len(dataVBOT3hr[1]['V']), 8):
        # On the left, jj is divided by 8 so it increases by 1 (each day)
        # On the right, 8 is added to jj to include all 8 daily data points in the mean
        vbot[ii][jj//8] = dataVBOT3hr[ii]['V'][jj:jj+8].mean(dim='time').values
        ubot[ii][jj//8] = dataUBOT3hr[ii]['U'][jj:jj+8].mean(dim='time').values
        

In [23]:
# Put the daily data back into a dataset. Use the daily prect time stamps 
# Initialize holding list
dataVBOT = [None] * (len(hcastDayStr)+1)
dataUBOT = [None] * (len(hcastDayStr)+1)
# Loop through list and create datasets
for ii in range(1, len(vbot) ):
    dataVBOT[ii] = xr.Dataset({'v':(['time','lat','lon'], vbot[ii])}, 
                          coords={'time':dataPRECT[ii]['time'], 'lat':dataVBOT3hr[1]['lat'], 'lon':dataVBOT3hr[1]['lon']})
    dataUBOT[ii] = xr.Dataset({'u':(['time','lat','lon'], ubot[ii])}, 
                          coords={'time':dataPRECT[ii]['time'], 'lat':dataVBOT3hr[1]['lat'], 'lon':dataVBOT3hr[1]['lon']})
